In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from imblearn.over_sampling import SMOTE

# Veriyi Yükle ve Hazırla (Önceki işlemlerin aynısı)
df = pd.read_csv('../data/ai4i2020.csv')
type_map = {'L': 0, 'M': 1, 'H': 2}
df['Type'] = df['Type'].map(type_map)

# Feature Engineering
df['Power_W'] = df['Torque [Nm]'] * df['Rotational speed [rpm]'] * (2 * np.pi / 60)
df['Temp_Diff'] = df['Process temperature [K]'] - df['Air temperature [K]']
df['Strain'] = df['Torque [Nm]'] * df['Tool wear [min]']

# Split ve SMOTE
drop_cols = ['UDI', 'Product ID', 'Machine failure', 'TWF', 'HDF', 'PWF', 'OSF', 'RNF']
X = df.drop(drop_cols, axis=1)
y = df['Machine failure']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

print("Veri Hazır. Optimizasyona Başlanıyor...")

Veri Hazır. Optimizasyona Başlanıyor...


In [2]:
# Denenecek Parametreler
param_grid = {
    'n_estimators': [100, 200],        # Ağaç sayısı
    'max_depth': [None, 10, 20],       # Ağaç derinliği (Çok derin olursa ezberler)
    'min_samples_split': [2, 5],       # Dallanma için gereken minimum örnek
    'class_weight': ['balanced', None] # Sınıf ağırlığı kullanılsın mı?
}


rf = RandomForestClassifier(random_state=42)

# Grid Search'ü Başlat (3 Katlı Çapraz Doğrulama ile)
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, 
                           cv=3, n_jobs=-1, scoring='recall', verbose=2)

grid_search.fit(X_train_resampled, y_train_resampled)

# En İyi Sonuçlar
print(f"En İyi Parametreler: {grid_search.best_params_}")
print(f"En İyi Recall Skoru (Eğitimde): {grid_search.best_score_:.4f}")

Fitting 3 folds for each of 24 candidates, totalling 72 fits
En İyi Parametreler: {'class_weight': None, 'max_depth': None, 'min_samples_split': 2, 'n_estimators': 200}
En İyi Recall Skoru (Eğitimde): 0.9917


In [3]:
# En iyi modeli al
best_model = grid_search.best_estimator_

# Tahmin
y_pred = best_model.predict(X_test)

print("\n=== Optimize Edilmiş Model Sonuçları ===")
print(classification_report(y_test, y_pred))

# Karşılaştırma için Recall
from sklearn.metrics import recall_score
print(f"Final Recall Skoru: {recall_score(y_test, y_pred):.4f}")


=== Optimize Edilmiş Model Sonuçları ===
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      1932
           1       0.70      0.85      0.77        68

    accuracy                           0.98      2000
   macro avg       0.85      0.92      0.88      2000
weighted avg       0.98      0.98      0.98      2000

Final Recall Skoru: 0.8529


## Model Optimizasyonu ve Hiperparametre Ayarı Raporu 

Bu bölümde, önceki aşamada (Feature Engineering + SMOTE) elde ettiğimiz başarılı Random Forest modelinin performansını maksimize etmek ve modelin kararlılığını (stability) artırmak amacıyla **Grid Search Cross-Validation (GridSearchCV)** yöntemi uygulanmıştır.

### 1. Optimizasyon Stratejisi
Modelin varsayılan (default) parametreleri yerine, veriye en uygun parametre setini bulmak için şu hiperparametre uzayı taranmıştır:

* **`n_estimators` (Ağaç Sayısı):** [100, 200] -> Daha fazla ağaç genelde daha kararlı sonuç verir.
* **`max_depth` (Maksimum Derinlik):** [None, 10, 20] -> Modelin ezberlemesini (Overfitting) önlemek için derinlik sınırı test edilmiştir.
* **`min_samples_split`:** [2, 5] -> Dallanma kriteri.
* **`class_weight`:** ['balanced', None] -> Dengesizliğe karşı ek bir önlem olarak test edilmiştir.

### 2. En İyi Parametreler (Best Parameters)
Yapılan 3 katlı çapraz doğrulama (3-Fold CV) sonucunda en iyi performansı veren kombinasyon şu olmuştur:
* **Ağaç Sayısı:** 200 (Modelin daha sağlam genelleme yapmasını sağladı)
* **Derinlik:** None (Ağaçların serbestçe büyümesine izin verildi)
* **Class Weight:** None (SMOTE zaten dengeyi sağladığı için ek ağırlığa gerek duyulmadı)

### 3. Sonuç ve Karşılaştırma
Optimize edilmiş modelin Test Seti üzerindeki performansı:

* **Final Recall Skoru:** **0.8529** (%85.3)
* **Önceki Skor (Manuel Model):** ~%85.0
* **Yorum:** Hiperparametre optimizasyonu sonucunda skorda dramatik bir artış yerine **stabilizasyon** sağlanmıştır. Bu durum, önceki aşamada kurduğumuz Feature Engineering ve SMOTE yapısının zaten çok güçlü olduğunu ve modelin doygunluk noktasına (saturation point) ulaştığını göstermektedir.

### Karar
Modelimiz, **%85 Arıza Yakalama (Recall)** ve **%98 Genel Doğruluk (Accuracy)** ile "Production" (Canlı) ortamı için optimize edilmiş ve onaylanmıştır. Ekstra karmaşıklık getirecek (Örn: Boosting modelleri) başka bir modele geçilmesine gerek görülmemiştir.